In [1]:
import copy
import itertools

In [2]:
def gcd(a, b):
    if b == 0:
        return a
    else:
        return gcd(b, a % b)
    
def lcm(a, b):
    return a * b // gcd(a, b)

In [3]:
def egcd(a, b):
    if 0 == b:
        return 1, 0, a
    x, y, q = egcd(b, a % b)
    x, y = y, (x - a // b * y)
    return x, y, q
 
 
def custom_chinese_remainder(a,b):
    nod = gcd(a[0],b[0])
    nok = lcm(a[0],b[0])
    a0,b0 = (a[0] // nod, b[0]) if a[0] <= b[0] else (a[0], b[0] // nod)
    m1, m2 = nok//a0, nok//b0
    m1i, m2i = egcd(m1, a0)[0], egcd(m2, b0)[0] 
    x = m1 * m1i * a[1] + m2 * m2i * b[1]
    x %= nok
    return (nok, max(a[1],b[1]) + (x - max(a[1],b[1]) % nok) % nok)

def apply_chinese_remainder(a,b):
    shift = (a[1]-b[1]) // a[0]
    r = (a[1]-b[1]) % a[0]
    nok, first = custom_chinese_remainder((a[0],r),(b[0],0))
    return (nok//a[0], (first - r) // a[0] - shift)

In [4]:
class VariablesSet:
    def __init__(self, items=[]):
        self.s = set()
        self.m = dict()
        for i in items:
            self.addItem(i)

    def addItem(self, value):
        #print(f'adding {value}')
        if isinstance(value, tuple) and len(value)==2 and isinstance(value[1], tuple) and isinstance(value[0], str):
            vn = value[0]
            va = value[1][0]
            vb = value[1][1]
            if vn not in self.m:
                self.m[vn] = dict()
            if va == 0:
                
                for a,bb in self.m[vn].items():
                    if a == 0:
                        continue
                    else:
                        for b in bb:
                            if vb>=b and ((vb - b) % a == 0):
                                return
                            if vb == b-a:
                                newBorder = b-a
                                while 0 in self.m[vn] and (newBorder-a) in self.m[vn][0]:
                                    newBorder-=a
                                    self.removeItem((vn,(0,newBorder)))
                                self.m[vn][a].remove(b)
                                self.m[vn][a].add(newBorder)
                                return
                    
                if 0 not in self.m[vn]:
                    self.m[vn][0] = set()
                self.m[vn][0].add(vb)
            else:
                for a,bb in self.m[vn].items():
                    if a == 0:
                        continue
                    else:
                        nok = lcm(a, va)
                        nod = gcd(a, va)
                        for b in list(bb):
                            if (vb - b) % nod == 0:
                                if nod == va:
                                    bb.remove(b)
                                    if len(bb) == 0:
                                        del self.m[vn][a]
                                        if len(self.m[vn])==0:
                                            del self.m[vn]
                                    self.addItem((vn,(va,vb)))
                                    for i in range(b,vb,a):
                                        self.addItem((vn,(0,i)))
                                    return
                                if nok == va:
                                    for i in range(vb,b,va):
                                        self.addItem((vn,(0,i)))
                                else:
                                    for i in range(0,nok//va):
                                        self.addItem((vn,(nok,vb+va*i)))
                                return
                # тут уже точно нет пересечений
                if vn in self.m and 0 in self.m[vn]:
                    for b in list(self.m[vn][0]):
                        if b>= vb and (b - vb) % va == 0:
                            self.removeItemInternal((vn,(0,b)))
                while vn in self.m and 0 in self.m[vn] and (vb-va) in self.m[vn][0]:
                    vb-=va
                    self.removeItemInternal((vn,(0,vb)))
                if va not in self.m[vn]:
                    self.m[vn][va] = set()
                self.m[vn][va].add(vb)

                if va == 1:
                    return
                if len(self.m[vn][va]) == va:
                    bb = set(self.m[vn][va])
                    del self.m[vn][va]
                    if len(self.m[vn])==0:
                        del self.m[vn]
                    start = max(bb)
                    self.addItem((vn,(1,start)))
                    for b in bb:
                        for i in range(b,start,va):
                            self.addItem((vn,(0,i)))
                else:
                    c = dict()
                    for b in self.m[vn][va]:
                        s = ((vb - b) % va + va) % va
                        if s != 0 and va % s == 0:
                            if s not in c:
                                c[s]=1
                            c[s]+=1
                    for k,v in c.items():
                        if va % k == 0 and va // k == v:
                            newb = max([b for b in self.m[vn][va] if (vb - b) % va == 0 or va % ((vb - b) % va) == 0])
                            for b in list(self.m[vn][va]):
                                s = (vb - b) % k
                                if s == 0:
                                    self.m[vn][va].remove(b)
                                    if len(self.m[vn][va]) == 0:
                                        del self.m[vn][va]
                                        if len(self.m[vn])==0:
                                            del self.m[vn]
                                    for i in range(b,newb,va):
                                        self.addItem((vn,(0,i)))
                            self.addItem((vn,(k, newb)))
                    


        elif isinstance(value, str):
            self.s.add(value)
        else:
            print(f'invalid Variable {value} in VariablesSet addItem')


    def removeItem(self, value):
        self.removeItemInternal(value)
        if isinstance(value, tuple) and value[0] in self.m and len(self.m[value[0]].keys())==0:
            del self.m[value[0]]

    def removeItemInternal(self, value):
        #print(f'removing {value}')
        if isinstance(value, tuple) and len(value)==2 and isinstance(value[1], tuple) and isinstance(value[0], str):
            vn = value[0]
            va = value[1][0]
            vb = value[1][1]
            if vn not in self.m:
                return
            if va == 0:
                for a,bb in self.m[vn].items():
                    if a == 0:
                        continue
                    else:
                        for b in bb:
                            if vb>=b and ((vb - b) % a == 0):
                                self.m[vn][a].remove(b)
                                self.m[vn][a].add(vb+a)
                                for i in range(b,vb,a):
                                    self.addItem((vn,(0,i)))
                                return
                if 0 in self.m[vn]:
                    self.m[vn][0].discard(vb)
                    if len(self.m[vn][0]) == 0:
                        del self.m[vn][0]
            else:
                if 0 in self.m[vn]:
                    newbb = set()
                    for b in self.m[vn][0]:
                        if b<vb or (b-vb) % va != 0:
                            newbb.add(b)
                    if len(newbb) == 0:
                        del self.m[vn][0]
                    else:
                        self.m[vn][0] = newbb
                for a,bb in self.m[vn].copy().items():
                    if a == 0:
                        continue
                    else:
                        nok = lcm(a, va)
                        nod = gcd(a, va)
                        for b in list(bb):
                            if (vb - b) % nod == 0:
                                if nod == va:
                                    if a in self.m[vn]:
                                        self.m[vn][a].remove(b)
                                    if len(self.m[vn][a]) == 0:
                                        del self.m[vn][a]
                                    for i in range(b,vb,a):
                                        self.addItem((vn,(0,i)))
                                    continue
                                if nok == va:
                                    self.m[vn][a].remove(b)
                                    if len(self.m[vn][a]) == 0:
                                        del self.m[vn][a]
                                    for i in range(b,b+va,a):
                                        if (vb-i) % va != 0:
                                            self.addItem((vn,(va,i)))
                                    for i in range(vb-va,b,-va):
                                        self.addItem((vn,(0,i)))
                                else:
                                    for i in range(0,nok//va):
                                        self.removeItemInternal((vn,(nok,vb+va*i)))
                                continue
        elif isinstance(value, str):
            self.s.discard(value)
        else:
            print(f'invalid Variable {value} in VariablesSet removeItem')


    def add(self, values):
        vlist = values
        if not isinstance(values,list):
            vlist = [values]
        for v in vlist:
            self.addItem(v)

    def update(self, vset):
        self.s.update(vset.s)
        for k,v in vset.m.items():
            for a,bb in v.items():
                for b in bb:
                    self.addItem((k, (a,b)))

    def union(self, vset):
        result = copy.deepcopy(self)
        result.update(vset)
        return result

    def intersection(self, vset):
        result = VariablesSet()
        result.s = self.s.intersection(vset.s)
        for k,v in self.m.items():
            if k not in vset.m:
                continue
            for a,bb in v.items():
                if a==0:
                    for va,vbb in vset.m[k].items():
                        if va==0:
                            result.add([(k,(0,b)) for b in bb.intersection(vbb)])
                        else:
                            for vb in vbb:
                                result.add([(k,(0,b)) for b in bb if b >= vb and (b-vb) % va == 0])
                else:
                    for b in bb:
                        for va,vbb in vset.m[k].items():
                            if va==0:
                                result.add([(k,(0,vb)) for vb in vbb if vb >= b and (b-vb) % a == 0])
                            else:
                                for vb in vbb:
                                    d = gcd(a,va)
                                    if (vb - b) % d == 0:
                                        result.add((k, custom_chinese_remainder((a,b),(va,vb))))
        return result


    def intersection_update(self, vset):
        res = self.intersection(vset)
        self.s = res.s
        self.m = res.m
        #TODO self = self.intersection(vset)?


    
    def difference_update(self, vset):
        self.s.difference_update(vset.s)
        for k,v in vset.m.items():
            for a,bb in v.items():
                for b in bb:
                    self.removeItem((k, (a,b)))

    def difference(self, vset):
         result = copy.deepcopy(self)
         result.difference_update(vset)
         return result

    def empty(self):
        if self.s or self.m:
            return False
        else:
            return True

    def clear(self):
        self.s.clear()
        self.m.clear()


    def isinValue(self, value):
        #print(f'isin {value}')
        if isinstance(value, tuple) and len(value)==2 and isinstance(value[1], tuple) and isinstance(value[0], str):
            vn = value[0]
            va = value[1][0]
            vb = value[1][1]
            if vn in self.m:
                if va == 0:
                    for a,bb in self.m[vn].items():
                        if a == 0:
                            if vb in bb:
                                return True
                        else:
                            for b in bb:
                                if vb >= b and (vb-b) % a == 0:
                                    return True
                else:
                    for a,bb in self.m[vn].items():
                        if a == 0 and vb in bb:
                            infb = vb + va
                            while infb in bb:
                                infb += va
                            return self.isinValue((vn,(va,infb)))
                        nok = lcm(a, va)
                        nod = gcd(a, va)
                        for b in bb:
                            if vb>=b and (vb - b) % nod == 0:
                                if nok == va:
                                    return True
            return False
        elif isinstance(value, str):
            if value not in self.s:
                return False
            return True
        else:
            return False

    def isin(self, values):
        vlist = values
        if not isinstance(values,list):
            vlist = [values]
        for value in vlist:
            if not self.isinValue(value):
                return False
        return True
            
    def isinVS(self, vset):
        for value in vset:
            if not self.isinValue(value):
                return False
        return True
    
    def expand(self, step=1):
        for k in self.m:
            if 0 in self.m[k]:
                self.addItem((k,(step, min(self.m[k][0]))))


    def names(self):
        yield from self.s
        for k in self.m:
            yield k

    def __iter__(self):
        yield from self.s
        
        for k, v in self.m.items():
            for a,bb in v.items():
                for b in bb:
                    yield (k, (a,b))



    def __repr__(self):
        return '{' + ', '.join(
                            sorted(self.s)
                            + [ ', '.join([ k + '[' +','.join([str(b) for b in sorted(bb)])+ ']' if a==0 else ', '.join([k + '['+('' if a == 1 else str(a))+'i+'+str(b)+']' for b in sorted(bb)]) for a,bb in v.items()])
                            for k,v in self.m.items()
                            ]
                        ) + '}'
    def __str__(self):
        return '{' + ', '.join(
                            sorted(self.s)
                            + [ ', '.join([ k + '[' +','.join([str(b) for b in sorted(bb)])+ ']' if a==0 else ', '.join([k + '['+('' if a == 1 else str(a))+'i+'+str(b)+']' for b in sorted(bb)]) for a,bb in v.items()])
                            for k,v in self.m.items()
                            ]
                        ) + '}'

In [5]:
class SOperation:
    def __init__(self, label:str, ins:list, outs:list, psi = None, auto = False, usage = None, scope = None):
        self.label = label
        self.ins = ins
        self.outs = outs
        self.psi = psi
        self.auto = auto
        self.usage = usage
        self.scope = scope

    def __repr__(self):
        return self.label + (('(psi(auto): ' if self.auto else '(psi: ') + str(self.psi) + ')' if self.psi is not None else '') +'(ins:'+ str(self.ins) + ', outs:' + str(self.outs)  + ')'
    def __str__(self):
        return self.label + (('(psi(auto): ' if self.auto else '(psi: ') + str(self.psi) + ')' if self.psi is not None else '') +'(ins:'+ str(self.ins) + ', outs:' + str(self.outs)  + ')'

In [6]:
class MOperation:
    def __init__(self, label:str, ins:list, outs:list, scope:VariablesSet):
        self.label = label
        self.ins = ins
        self.outs = outs
        self.scope = scope
        
    def __repr__(self):
        return self.label + '['+str(self.scope)+']' +'(ins:'+ str(self.ins) + ', outs:' + str(self.outs)  + ')'
    def __str__(self):
        return self.label + '['+str(self.scope)+']' +'(ins:'+ str(self.ins) + ', outs:' + str(self.outs)  + ')'

In [7]:
class SVariable:
    def __init__(self, label, entry, comp):
        self.label = label
        self.entry = entry
        self.comp = comp
    def __repr__(self):
        return self.label + '(entry:'+ str(self.entry) + ', comp:' + str(self.comp)  + ')'
    def __str__(self):
        return self.label + '(entry:'+ str(self.entry) + ', comp:' + str(self.comp)  + ')'

In [8]:
class MVariable:
    def __init__(self, label, entry, comp):
        self.label = label
        self.entry = entry
        self.comp = comp
    def __repr__(self):
        return self.label + '(entry:'+ str(self.entry) + ', comp:' + str(self.comp)  + ')'
    def __str__(self):
        return self.label + '(entry:'+ str(self.entry) + ', comp:' + str(self.comp)  + ')'

In [9]:
class Model:
    def __init__(self, v=dict(), o=dict()):
        self.v = v
        self.o = o
    def __repr__(self):
        return str(self.v) +'\n'+ str(self.o)
    def __str__(self):
        return str(self.v) +'\n'+ str(self.o)

In [10]:
class Term:
    def __init__(self, top):
        self.top = top
        self.ins = list()

    def __getitem__(self, i):
        res = Term((self.top[0],(0,self.top[1][0]*i + self.top[1][1])))
        for t in self.ins:
            if isinstance(t, Term):
                res.ins.append(t[i])
            else:
                res.ins.append((t[0],(0,t[1][0]*i + t[1][1])))
        return res

    def __repr__(self):
        return self.top[0] + '['+str(self.top[1])+']' + '(' + ', '.join([ str(x) if isinstance(x,Term) else x[0]+'['+str(x[1])+']' for x in self.ins]) + ')'
    def __str__(self):
        return self.top[0] + '['+str(self.top[1])+']' + '(' + ', '.join([ str(x) if isinstance(x,Term) else x[0]+'['+str(x[1])+']' for x in self.ins]) + ')'
    
    

In [11]:
def termIns(term, i = 0):
    if isinstance(term, Term):
        vars = VariablesSet()
        for t in term.ins:
            tvars = termIns(t, i)
            vars.update(tvars)
        return vars
    else:
        if isinstance(term, tuple) and i>=0:
            return VariablesSet([(term[0],(0,term[1][0]*i+term[1][1]))])
        else:
            return VariablesSet([term])

In [12]:
def termVars(term, model:Model, i = 0):
    if isinstance(term, Term):
        vars = VariablesSet()

        for ov in model.o[term.top[0]].outs:
            if not isinstance(ov,tuple):
                vars.addItem(ov)
            else:
                if i < 0:
                    vars.addItem((ov[0],(term.top[1][0]*ov[1][0],ov[1][0]*term.top[1][1]+ov[1][1])))
                else:
                    vars.addItem((ov[0],(0,ov[1][0]*(term.top[1][0]*i + term.top[1][1])+ov[1][1])))
        for t in term.ins:
            tvars = termVars(t, model, i)
            vars.update(tvars)
        return vars
    else:
        if isinstance(term, tuple) and i>=0:
            return VariablesSet([(term[0],(0,term[1][0]*i+term[1][1]))])
        else:
            return VariablesSet([term])

In [13]:
def termOutput(term, model, i = 0):
    if isinstance(term, Term):
        vars = VariablesSet()
        for ov in model.o[term.top[0]].outs:
            if not isinstance(ov,tuple):
                vars.addItem(ov)
            else:
                if i < 0:
                    vars.addItem((ov[0],(term.top[1][0]*ov[1][0],ov[1][0]*term.top[1][1]+ov[1][1])))
                else:
                    vars.addItem((ov[0],(0,ov[1][0]*(term.top[1][0]*i + term.top[1][1])+ov[1][1])))
        for t in term.ins:
            tvars = termOutput(t, model, i)
            vars.update(tvars)
        return vars
    else:
        return VariablesSet()

In [14]:
def checkOpEntrExistence(model:Model, toRemove:VariablesSet, op1, op2):
    opl = op1[0]
    scope = model.o[opl].scope.difference(toRemove)
    if scope.isinValue((opl,(abs(op2[1][1]-op1[1][1]), min(op1[1][1], op2[1][1])))):
        return True
    else:
        return False

In [15]:
def buildIdenticalTerms(top1, top2, model:Model, L:dict[str,list[tuple[int,int]]], toRemove:VariablesSet, signify1=dict(), signify2=dict()):
    opl = top1[0]
    i1, i2 = top1[1][1], top2[1][1]
    shift = abs(i1-i2)
    if opl != top2[0] or shift == 0:
        return []
    if not checkOpEntrExistence(model, toRemove, top1, top2):
        return []
    l1, l2 = top1[1][0], top2[1][0]
    res = []
    print(f'building terms for {opl}: {l1},{i1} and {l2},{i2}')
    if l1>=l2:
        terms = []
        for var, iv in enumerate(model.o[opl].ins):
            terms.append([])
            if not isinstance(iv, tuple):
                terms[var].append((iv,iv))
                continue
            ivl = iv[0]
            if iv[1][0] == 0:
                terms[var].append((iv,iv))
                continue
            else:
                ivi1 = iv[1][0]*i1 + iv[1][1]
                ivi2 = iv[1][0]*i2 + iv[1][1]
                found = False
                for fl in model.v[ivl].comp:
                    if fl not in L:
                        continue
                    ls = sorted(L[fl], key=lambda x: x[0])
                    for f1 in ls:
                        for f2 in ls:
                            if f1[1]==f2[1]:
                                continue
                            if f2[0]>f1[0]:
                                break
                            if fl in signify1 and (f1[1],iv) in signify1[fl]:
                                continue
                            if fl in signify2 and (f2[1],iv) in signify2[fl]:
                                continue
                            
                            canBeApplyed1, canBeApplyed2 = False, False
                            for ov in model.o[fl].outs:
                                if (ov[1][0]*f1[1] + ov[1][1]) == ivi1:
                                    canBeApplyed1 = True
                                if (ov[1][0]*f2[1] + ov[1][1]) == ivi2:
                                    canBeApplyed2 = True
                            if not (canBeApplyed1 and canBeApplyed2):
                                continue

                            if fl not in signify1:
                                signify1[fl] = set()
                            if fl not in signify2:
                                signify2[fl] = set()
                            signify1[fl].add((f1[1],iv))
                            signify2[fl].add((f2[1],iv))
                            varinterms = buildIdenticalTerms((fl,f1), (fl,f2), model, L, toRemove, signify1, signify2)
                            terms[var]+=varinterms
                            signify1[fl].remove((f1[1],iv))
                            signify2[fl].remove((f2[1],iv))
                            if len(varinterms):
                                found = True
                if not found:
                    divi = abs(ivi1-ivi2)
                    terms[var].append(((ivl,(divi,ivi1)),(ivl,(divi,ivi2))))
        combinations = list(itertools.product(*terms))
        for comb in combinations:
            t1 = Term((opl,(shift,i1)))
            t2 = Term((opl,(shift,i2)))
            t1.ins = [c[0] for c in comb]
            t2.ins = [c[1] for c in comb]
            res.append((t1,t2))
            #print(f'res.append by {(t1,t2)}')
    if l1<=l2:
        terms = []
        for var, iv in enumerate(model.o[opl].ins):
            terms.append([])
            if not isinstance(iv, tuple):
                terms[var].append((iv,iv))
                continue
            ivl = iv[0]
            if iv[1][0] == 0:
                terms[var].append((iv,iv))
                continue
            else:
                ivi1 = iv[1][0]*i1 + iv[1][1]
                ivi2 = iv[1][0]*i2 + iv[1][1]
                found = False
                for fl in model.v[ivl].comp:
                    if fl not in L:
                        continue
                    ls = sorted(L[fl], key=lambda x: x[0], reverse=True)
                    for f1 in ls:
                        for f2 in ls:
                            if f1[1]==f2[1]:
                                continue
                            if f2[0]<f1[0]:
                                break
                            if fl in signify1 and (f1[1],iv) in signify1[fl]:
                                continue
                            if fl in signify2 and (f2[1],iv) in signify2[fl]:
                                continue

                            canBeApplyed1, canBeApplyed2 = False, False
                            for ov in model.o[fl].outs:
                                if (ov[1][0]*f1[1] + ov[1][1]) == ivi1:
                                    canBeApplyed1 = True
                                if (ov[1][0]*f2[1] + ov[1][1]) == ivi2:
                                    canBeApplyed2 = True
                            if not (canBeApplyed1 and canBeApplyed2):
                                continue

                            if fl not in signify1:
                                signify1[fl] = set()
                            if fl not in signify2:
                                signify2[fl] = set()
                            signify1[fl].add((f1[1],iv))
                            signify2[fl].add((f2[1],iv))
                            varinterms = buildIdenticalTerms((fl,f1), (fl,f2), model, L, toRemove, signify1, signify2)
                            terms[var]+=varinterms
                            signify1[fl].remove((f1[1],iv))
                            signify2[fl].remove((f2[1],iv))
                            if len(varinterms):
                                found = True
                if not found:
                    divi = abs(ivi1-ivi2)
                    terms[var].append(((ivl,(divi,ivi1)),(ivl,(divi,ivi2))))
        combinations = list(itertools.product(*terms))
        for comb in combinations:
            t1 = Term((opl,(shift,i1)))
            t2 = Term((opl,(shift,i2)))
            t1.ins = [c[0] for c in comb]
            t2.ins = [c[1] for c in comb]
            res.append((t1,t2))
            #print(f'res.append by {(t1,t2)}')

    return res



In [16]:
def checkSelfReplenish(ins:VariablesSet, term):
    if isinstance(term, Term):
        res = VariablesSet()
        for t in term.ins:
            res.update(checkSelfReplenish(ins, t))
        return res
    else:
        if not ins.isinValue((term[0], (0,term[1][1]))):
            print(f'{term} is not in {ins}')
            return VariablesSet([term])
        return VariablesSet()

In [17]:
def addTermOperations(ls : VariablesSet , term):
    #print(f'addTermOperations of {term}')
    if isinstance(term, Term):
        ls.add(term.top)
        for t in term.ins:
            addTermOperations(ls, t)
    

In [18]:
def canApply(op:MOperation, V: VariablesSet):
    scope = copy.deepcopy(op.scope)
    for iv in op.ins:
        if isinstance(iv, tuple):
            if iv[1][0]==0:
                if not V.isin(iv):
                    return VariablesSet()
            elif iv[0] in V.m:
                entrScope = VariablesSet()
                for a,bb in V.m[iv[0]].items():
                    if a==0:
                        for b in bb:
                            if (b-iv[1][1]) % iv[1][0] == 0:
                                entrScope.add((op.label,(a, (b-iv[1][1])//iv[1][0])))
                    else:
                        d = gcd(a,iv[1][0])
                        for b in bb:
                            if (iv[1][1] - b) % d == 0:
                                entrScope.addItem((op.label,apply_chinese_remainder(iv[1],(a,b))))
                scope.intersection_update(entrScope)
                if scope.empty():
                    return scope
            else:
                return VariablesSet()
        else:
            if iv not in V.s:
                return VariablesSet()
    return scope

In [19]:
def canApplyBackward(op, V: VariablesSet):
    scope = VariablesSet()
    if isinstance(op, MOperation):
        for ov in op.outs:
            if not isinstance(ov, tuple):
                if V.isin(ov):
                    return op.scope
                continue
            if ov[1][0]==0:
                continue
            if not ov[0] in V.m:
                continue
            for a,bb in V.m[ov[0]].items():
                if a==0:
                    for b in bb:
                        if (b-ov[1][1]) % ov[1][0] == 0:
                            scope.add((op.label,(a,(b-ov[1][1])//ov[1][0])))
                else:
                    for b in bb:
                        d = gcd(a,ov[1][0])
                        if (ov[1][1] - b) % d == 0:
                            scope.addItem((op.label,apply_chinese_remainder(ov[1],(a,b))))
        scope.intersection_update(op.scope)
    else:
        for ov in op.outs:
            if not isinstance(ov, tuple):
                if V.isin(ov):
                    return VariablesSet([op.label])
                continue
            if not V.intersection(VariablesSet([(ov)])).empty():
                return VariablesSet([op.label])
    return scope

In [20]:
def operationUsage(model:Model, term, ready:list):
    if isinstance(term,Term):
        for t in term.ins:
            operationUsage(model,t,ready)
        for i, r in enumerate(ready):
            if term.top[0] == r[0]:
                if term.top[1][0] == r[1][0] and (term.top[1][1]-r[1][1]) % r[1][0] == 0:
                    ready[i] = (r[0],(r[1][0],min(term.top[1][1],r[1][1])))
                    return
        ready.append(term.top)

In [21]:
def calculateUsage(op:SOperation, model:Model):
    ready = []
    operationUsage(model, op.psi, ready)
    return ready

In [22]:
psiid = 0

In [23]:
def handleVariable(op, model:Model, L, Lprev, Vprimei:VariablesSet, Vprimenew:VariablesSet, Fi:VariablesSet, toRemove:VariablesSet, newPsiOperations, primelevel):
                    global psiid
                    if not isinstance(op, tuple):
                        print(f'simple operations in Fprimei are ignored')
                        return
                    if op[1][0] == 0:
                        if toRemove.isin(op):
                            return
                        if op[0] not in L:
                            L[op[0]] = [(primelevel,op[1][1])]
                        else:
                            psicreated = False
                            for opj in sorted(L[op[0]], key=lambda x: abs(op[1][1]-x[1]), reverse=False):
                                if toRemove.isin((op[0], (0,opj[1]))):
                                    continue
                                op1, op2 = ((op[0],opj),(op[0],(primelevel,op[1][1]))) if opj[1] < op[1][1] else ((op[0],(primelevel,op[1][1])),(op[0],opj))
                                if not checkOpEntrExistence(model, toRemove, op1, op2):
                                    print(f'ingnored {op[0]}: {op1[1]} and {op2[1]}')
                                    continue
                                print(f'\t\ttrying to build terms for {op[0]}: {op1[1]} and {op2[1]}')
                                terms = buildIdenticalTerms(op1, op2, model, Lprev, toRemove)
                                step = op2[1][1]-op1[1][1]
                                for t1,t2 in terms:
                                    print(f'{t1} and {t2} are identical with step = {step}')
                                    psi = t1
                                    tvars = termVars(psi, model, 0)
                                    inVars = termIns(psi, 0)
                                    tins = termIns(psi, 1)
                                    print(f'new psi is {psi}, ins[0]:{inVars}, ins[1]:{tins}, vars:{tvars}')
                                    infvars = checkSelfReplenish(tvars,t2)
                                    if Vprimei.isinVS(infvars):
                                        print('self replenish: True')
                                        psicreated = True
                                        newVars = termOutput(psi, model, -1)
                                        print(f'newVars: {newVars}')
                                        newPsiOperations.append(SOperation(label='psi'+str(psiid),
                                                                           ins=inVars.union(infvars),
                                                                           outs=newVars,
                                                                           psi=psi))
                                        addTermOperations(toRemove, psi)
                                        print(f'TOREMOVE is {toRemove}')
                                        Vprimenew.update(newVars)
                                        Fi.addItem('psi'+str(psiid))
                                        psiid+=1
                                        break
                                    else:
                                        print('self replenish: False')
                                if psicreated:
                                    break
                            if not psicreated:
                                L[op[0]].append((primelevel,op[1][1]))
                                print(f'\t\t{op[0]}[{(primelevel,op[1][1])}] added to L')
                    else:
                        opvs = VariablesSet([op])
                        intvs = opvs.intersection(toRemove)
                        if not intvs.empty():
                            opvs.difference_update(intvs)
                            for opop in opvs:
                                handleVariable(opop, model, L, Lprev, Vprimei, Vprimenew, Fi, toRemove, newPsiOperations, primelevel)
                            return
                        
                        opvs.difference_update(toRemove)
                        psi=Term(op)
                        for iv in model.o[op[0]].ins:
                            if isinstance(iv, tuple):
                                psi.ins.append((iv[0],(iv[1][0]*op[1][0],iv[1][1]+op[1][1]*iv[1][0])))
                            else:
                                psi.ins.append(iv)
                        print(f'automatic psi is {psi}')
                        newVars = termOutput(psi, model, -1)
                        print(f'newVars: {newVars}')
                        inVars = termIns(psi, -1)
                        newPsiOperations.append(SOperation(label='psi'+str(psiid),
                                                               ins=inVars,
                                                               outs=newVars,
                                                               psi=psi,
                                                               auto=True
                                                               ))
                        toRemove.add(op)
                        print(f'TOREMOVE is {toRemove}')
                        Vprimenew.update(newVars)
                        Fi.add('psi'+str(psiid))
                        psiid+=1

In [24]:
def ascending(model : Model, V_in: VariablesSet, maxprimelevel=100):
    level = 0
    Vi = copy.deepcopy(V_in)
    Fsum = VariablesSet()
    Fi = VariablesSet()
    Fprimei = VariablesSet()
    for k,v in model.o.items():
        if len(v.ins) == 0:
            if isinstance(v,MOperation):
                print('ERROR: massive operation has empty IN')
            else:
                for iv in v.outs:
                    Vi.add(iv)
                    print(f'{k} can be applied, {iv} was added to V_0')
                    Fsum.add(k)
    print('V_0 is', Vi)
    Vnew = copy.deepcopy(Vi)
    psiid = 0
    while True:
        Fi.clear()
        Fprimei.clear()
        ops = set()
        for vl in Vnew.names():
            ops.update(model.v[vl].entry)
        print(f'for {str(Vnew)} ops are {ops}')
    
        for opl in ops:
            op = model.o[opl]
            if isinstance(op, MOperation):
                res = canApply(op, Vi)
                res.difference_update(Fsum)
                if not res.empty():
                    Fprimei.update(res)
                    print(f'\t{str(res)} added to F\'_0')
            else:
                if Vi.isin(op.ins) and not Fsum.isin(opl):
                    Fi.add(opl)
                    print(f'\t{opl} added to F_{level}')

        # imitation algorithm
        if not Fprimei.empty():
            print('\nimitation start')
            newPsiOperations = []
            Vprimei = copy.deepcopy(Vi)
            Vprimenew = VariablesSet()
            Fprimesum = VariablesSet()
            primelevel = 0
            L = dict()
            print(f'\tF\'_{primelevel} is', Fprimei)

            while True:
                Vprimenew.clear()
                toRemove = VariablesSet()
                Lprev = copy.deepcopy(L)
                for op in Fprimei:
                    if isinstance(op, tuple) and op[1][0]==0:
                        continue
                    handleVariable(op, model, L, Lprev, Vprimei, Vprimenew, Fi, toRemove, newPsiOperations, primelevel)
                for op in Fprimei:
                    if isinstance(op, tuple) and op[1][0]==0:
                        handleVariable(op, model, L, Lprev, Vprimei, Vprimenew, Fi, toRemove, newPsiOperations, primelevel)

                Fprimesum.difference_update(toRemove)
                Fprimei.difference_update(toRemove)
                Fprimesum.update(Fprimei)
                for n in toRemove.names():
                    model.o[n].scope.difference_update(toRemove)
                    if n in L:
                        newLn=[]
                        for Ll,Li in L[n]:
                            if not toRemove.isin((n,(0,Li))):
                                newLn.append((Ll,Li))
                        L[n] = newLn

                for op in Fprimei:
                    for ov in model.o[op[0]].outs:
                        if isinstance(ov,tuple):
                                newvl=(ov[0], (op[1][0],ov[1][0]*op[1][1]+ov[1][1]))
                                if not Vprimei.isin(newvl):
                                    Vprimenew.add(newvl)
                                    addedstr=(newvl[0]+'['+str(newvl[1])+']')
                                    print(f'\t\t{addedstr} added to V\'_{primelevel+1} because {op} is in F\'_{primelevel}')
                        else:
                            Vprimenew.add(ov)
                            print(f'\t\t{ov} added to V\'_{primelevel+1} because {op} is in F\'_{primelevel}')


                print(f'\tL_{primelevel} is', L)
                primelevel+=1
                Vprimei.update(Vprimenew)
                print(f'\tV\'_{primelevel} is', Vprimei)
                Fprimesum.update(Fprimei)
                print(f'\tF\'sum_{primelevel} is', Fprimesum)
                Fprimei.clear()
                primeops = set()
                for vl in Vprimenew.names():
                    primeops.update(model.v[vl].entry)
                print(f'new primeops are {primeops}')

                for opl in primeops:
                    op = model.o[opl]
                    if isinstance(op, MOperation):
                        res = canApply(op, Vprimei)
                        if res.empty():
                            continue
                        res.difference_update(Fprimesum)
                        if not res.empty():
                            Fprimei.update(res)
                            print(f'\t\t{str(res)} added to F\'_{primelevel}')
                    else:
                        if Vi.isinVS(op.ins):
                            Fi.add(opl)
                            print(f'\t\t{opl} added to F\'_{primelevel}')

                if Fprimei.empty():
                    print(f'final Primesum is {Fprimesum}')
                    Fi.update(Fprimesum)
                    break
                if primelevel > maxprimelevel:
                    print("Reached max primelevel. Terminating...")
                    break
            print('imitation end\n')
            print(f'new Psi operations: {newPsiOperations}')
            for op in newPsiOperations:
                op.usage = calculateUsage(op, model)
            model.o = dict(model.o, **{x.label: x for x in newPsiOperations})
            for op in newPsiOperations:
                for v in op.ins:
                    model.v[v[0] if isinstance(v,tuple) else v].entry.append(op.label)
                for v in op.outs:
                    model.v[v[0] if isinstance(v,tuple) else v].comp.append(op.label)


        
        print(f'F_{level} is', Fi)
        Fsum.update(Fi)
        if Fi.empty():
            break

        Vnew.clear()
        for op in Fi:
            if isinstance(op, tuple):
                for vl in model.o[op[0]].outs:
                    if isinstance(vl, tuple):
                        newv = (vl[0],(0,op[1][1]*vl[1][0]+vl[1][1]))
                        if not Vi.isin(newv):
                            Vnew.add(newv)
                            print(f'\t{newv} added to V_{level+1} because {op} is in F_{level}')
                    else:
                        if not Vi.isin(vl):
                            Vnew.add(vl)
                            print(f'\t{vl} added to V_{level+1} because {op} is in F_{level}')
            else:
                for vl in model.o[op].outs:
                    if not Vi.isin(vl):
                        Vnew.add(vl)
                        addedstr=(vl[0]+'['+str(vl[1])+']') if isinstance(vl,tuple) else vl
                        print(f'\t{addedstr} added to V_{level+1} because {op} is in F_{level}')
        Vi.update(Vnew)
        level+=1
        print(f'V_{level} is', Vi)
        #print(str(model))

    return Fsum, Vi

            
            
                    

In [25]:
def descending(model:Model, W:VariablesSet, Fsum:VariablesSet):
    print("descending :O")
    Gi = VariablesSet()
    Hi = copy.deepcopy(W)
    Gsum = VariablesSet()
    Hsum = copy.deepcopy(W)
    level = 1;
    print(f'H_0 is', Hi)
    while True:
        Gi.clear()
        for v in Hi.names():
            for opl in model.v[v].comp:
                op = model.o[opl]
                res = canApplyBackward(op, Hi)
                res.intersection_update(Fsum)
                res.difference_update(Gsum)
                Gi.update(res)

        print(f'G_{level} is', Gi)
        if Gi.empty():
            break
        Hi.clear()
        for op in Gi:
            if isinstance(op, tuple):
                for v in model.o[op[0]].ins:
                    if isinstance(v,tuple):
                        if not Hsum.isinValue((v[0],(op[1][0]*v[1][0], v[1][0]*op[1][1]+v[1][1]))):
                            Hi.add((v[0],(op[1][0]*v[1][0], v[1][0]*op[1][1]+v[1][1])))
                    else:
                        if not Hsum.isinValue(v):
                            Hi.add(v)
            else:
                for v in model.o[op].ins:
                    if not Hsum.isinValue(v):
                        Hi.add(v)
        Gsum.update(Gi)
        Hsum.update(Hi)
        print(f'H_{level} is', Hi)
        level+=1

    return Gsum, Hsum



In [26]:
def filterModel(model:Model, operations:VariablesSet, variables:VariablesSet):
    for v in list(model.v.keys()):
        if v not in variables.names():
            del model.v[v]
        else:
            newComp = []
            for op in model.v[v].comp:
                if op in operations.names():
                    newComp.append(op)
            newEntry = []
            for op in model.v[v].entry:
                if op in operations.names():
                    newEntry.append(op)
            model.v[v].comp = newComp
            model.v[v].entry = newEntry

    for o in list(model.o.keys()):
        if o not in operations.names():
            del model.o[o]
        elif isinstance(model.o[o], MOperation):
            model.o[o].scope.intersection_update(operations)

In [27]:
def planCalc(model:Model, V:VariablesSet, W:VariablesSet,maxprimelevel=100):
    global psiid
    psiid = 0
    tmodel = copy.copy(model)
    Fsum, Vlast = ascending(tmodel, V, maxprimelevel)
    if not Vlast.isinVS(W):
        print('W cannot be calculated')
        return None
    Gsum, Hsum = descending(tmodel, W, Fsum)
    print('Gsum is', Gsum)
    print('Hsum is', Hsum)
    filterModel(tmodel, operations=Gsum, variables=Hsum)
    return tmodel

Примеры запуска

In [28]:
modelOperations=[
    SOperation('e', ['n'] , [('x',(0,1))]),
    MOperation('f', [('x',(1,0))] , [('y',(1,0))], VariablesSet([('f',(1,0))])),
    MOperation('g', [('y',(1,0))] , [('x',(1,1))], VariablesSet([('g',(1,0))])),
    MOperation('a', [('y',(1,0))] , [('z',(1,0))], VariablesSet([('a',(1,0))]))
    ]
modelVariables=[
    SVariable('n', ['e'], []),
    MVariable('x', ['f'], ['e','g']),
    MVariable('y', ['g','a'], ['f']),
    MVariable('z', [], ['a'])
]
simplemodel = Model(v={x.label : x for x in modelVariables},
              o={x.label : x for x in modelOperations})

print('\n\n'+str(simplemodel)+'\n\n')
simplemodelready4lunatranslation = planCalc(simplemodel,VariablesSet(['n']),VariablesSet([('x',(0,20)),('z',(0,20))]))
print('\n\n'+str(simplemodelready4lunatranslation)+'\n\n')



{'n': n(entry:['e'], comp:[]), 'x': x(entry:['f'], comp:['e', 'g']), 'y': y(entry:['g', 'a'], comp:['f']), 'z': z(entry:[], comp:['a'])}
{'e': e(ins:['n'], outs:[('x', (0, 1))]), 'f': f[{f[i+0]}](ins:[('x', (1, 0))], outs:[('y', (1, 0))]), 'g': g[{g[i+0]}](ins:[('y', (1, 0))], outs:[('x', (1, 1))]), 'a': a[{a[i+0]}](ins:[('y', (1, 0))], outs:[('z', (1, 0))])}


V_0 is {n}
for {n} ops are {'e'}
	e added to F_0
F_0 is {e}
	x[(0, 1)] added to V_1 because e is in F_0
V_1 is {n, x[1]}
for {x[1]} ops are {'f'}
	{f[1]} added to F'_0

imitation start
	F'_0 is {f[1]}
		y[(0, 1)] added to V'_1 because ('f', (0, 1)) is in F'_0
	L_0 is {'f': [(0, 1)]}
	V'_1 is {n, x[1], y[1]}
	F'sum_1 is {f[1]}
new primeops are {'a', 'g'}
		{a[1]} added to F'_1
		{g[1]} added to F'_1
		z[(0, 1)] added to V'_2 because ('a', (0, 1)) is in F'_1
		x[(0, 2)] added to V'_2 because ('g', (0, 1)) is in F'_1
	L_1 is {'f': [(0, 1)], 'a': [(1, 1)], 'g': [(1, 1)]}
	V'_2 is {n, x[1,2], y[1], z[1]}
	F'sum_2 is {f[1], a[1], g[

In [30]:
modelOperations=[
    SOperation('initf1', [] , [('f',(0,1))]),
    SOperation('initf2', [] , [('f',(0,2))]),
    SOperation('inits', [] , [('s',(0,0))]),
    MOperation('next', [('f',(1,1)),('f',(1,2))], [('f',(1,3))], VariablesSet([('next',(1,0))])),
    MOperation('sum', [('f',(1,1)),('s',(1,0))], [('s',(1,1))], VariablesSet([('sum',(1,0))])),
    SOperation('initfn', ['n',('f',(1,1))], ['fn']),
    SOperation('initsn', ['n',('s',(1,0))], ['sn'])
    ]
modelVariables=[
    SVariable('n', ['initf1','initf2','inits','initfn','initsn'], []),
    SVariable('fn', [], ['initfn']),
    SVariable('sn', [], ['initsn']),
    MVariable('f', ['next', 'sum','initfn'], ['initf1','initf2','next']),
    MVariable('s', ['sum','initsn'],['inits','sum'])
]
fibmodel = Model(v={x.label : x for x in modelVariables},
              o={x.label : x for x in modelOperations})
fibmodelready4lunatranslation = planCalc(fibmodel,VariablesSet(['n']),VariablesSet(['fn',('s',(0,9))]))
print('\n\n'+str(fibmodel)+'\n\n')
print('\n\n'+str(fibmodelready4lunatranslation)+'\n\n')

initf1 can be applied, ('f', (0, 1)) was added to V_0
initf2 can be applied, ('f', (0, 2)) was added to V_0
inits can be applied, ('s', (0, 0)) was added to V_0
V_0 is {n, f[1,2], s[0]}
for {n, f[1,2], s[0]} ops are {'initsn', 'inits', 'next', 'sum', 'initf2', 'initfn', 'initf1'}
	{next[0]} added to F'_0
	{sum[0]} added to F'_0

imitation start
	F'_0 is {next[0], sum[0]}
		f[(0, 3)] added to V'_1 because ('next', (0, 0)) is in F'_0
		s[(0, 1)] added to V'_1 because ('sum', (0, 0)) is in F'_0
	L_0 is {'next': [(0, 0)], 'sum': [(0, 0)]}
	V'_1 is {n, f[1,2,3], s[0,1]}
	F'sum_1 is {next[0], sum[0]}
new primeops are {'initfn', 'initsn', 'next', 'sum'}
		{next[1]} added to F'_1
		{sum[1]} added to F'_1
		trying to build terms for next: (0, 0) and (1, 1)
building terms for next: 0,0 and 1,1
next[(1, 0)](f[(1, 1)], f[(1, 2)]) and next[(1, 1)](f[(1, 2)], f[(1, 3)]) are identical with step = 1
new psi is next[(1, 0)](f[(1, 1)], f[(1, 2)]), ins[0]:{f[1,2]}, ins[1]:{f[2,3]}, vars:{f[1,2,3]}
self r

In [31]:
modelOperations=[
    SOperation('e', ['n'] , [('x',(0,1))]),
    MOperation('f', [('x',(1,0))] , [('y',(1,1))], VariablesSet([('f',(1,0))])),
    MOperation('g', [('y',(1,0))] , [('x',(1,1))], VariablesSet([('g',(1,0))])),
    MOperation('a', [('y',(1,0))] , [('z',(1,0))], VariablesSet([('a',(1,0))])),
    MOperation('b', [('x',(1,0))] , [('z',(1,0))], VariablesSet([('b',(1,0))]))
    ]
modelVariables=[
    SVariable('n', ['e'], []),
    MVariable('x', ['f','b'], ['e','g']),
    MVariable('y', ['g','a'], ['f']),
    MVariable('z', [], ['a','b'])
]
complicatedmodel = Model(v={x.label : x for x in modelVariables},
              o={x.label : x for x in modelOperations})

print('\n\n'+str(complicatedmodel)+'\n\n')
print('\n\n'+str(planCalc(complicatedmodel,VariablesSet(['n']),VariablesSet([('x',(0,11)),('z',(2,10))])))+'\n\n')



{'n': n(entry:['e'], comp:[]), 'x': x(entry:['f', 'b'], comp:['e', 'g']), 'y': y(entry:['g', 'a'], comp:['f']), 'z': z(entry:[], comp:['a', 'b'])}
{'e': e(ins:['n'], outs:[('x', (0, 1))]), 'f': f[{f[i+0]}](ins:[('x', (1, 0))], outs:[('y', (1, 1))]), 'g': g[{g[i+0]}](ins:[('y', (1, 0))], outs:[('x', (1, 1))]), 'a': a[{a[i+0]}](ins:[('y', (1, 0))], outs:[('z', (1, 0))]), 'b': b[{b[i+0]}](ins:[('x', (1, 0))], outs:[('z', (1, 0))])}


V_0 is {n}
for {n} ops are {'e'}
	e added to F_0
F_0 is {e}
	x[(0, 1)] added to V_1 because e is in F_0
V_1 is {n, x[1]}
for {x[1]} ops are {'f', 'b'}
	{f[1]} added to F'_0
	{b[1]} added to F'_0

imitation start
	F'_0 is {f[1], b[1]}
		y[(0, 2)] added to V'_1 because ('f', (0, 1)) is in F'_0
		z[(0, 1)] added to V'_1 because ('b', (0, 1)) is in F'_0
	L_0 is {'f': [(0, 1)], 'b': [(0, 1)]}
	V'_1 is {n, x[1], y[2], z[1]}
	F'sum_1 is {f[1], b[1]}
new primeops are {'g', 'a'}
		{g[2]} added to F'_1
		{a[2]} added to F'_1
		x[(0, 3)] added to V'_2 because ('g', (0

In [32]:
modelOperations=[
    SOperation('initx', [] , [('x',(0,0))]),
    MOperation('f', [('x',(1,0))] , [('x',(1,1)),('y',(2,0))], VariablesSet([('f',(1,0))])),
    MOperation('g', [('y',(2,0))] , [('y',(2,1))], VariablesSet([('g',(1,0))])),
    MOperation('h', [('x',(1,0)),('y',(1,0))] , [('z',(1,0))], VariablesSet([('h',(1,0))])),
    SOperation('initxn', ['n',('x',(1,0))] , ['xn']),
    SOperation('initw', ['xn'] , [('w',(0,10))]),
    MOperation('a', [('y',(3,0)),('w',(1,0))] , [('w',(1,1))], VariablesSet([('a',(1,0))]))
    ]

modelVariables=[
    SVariable('n', ['initxn'], []),
    SVariable('xn', ['initw'], ['initxn']),
    MVariable('x', ['f','h','initxn'], ['initx','f']),
    MVariable('y', ['g','h','a'], ['f','g']),
    MVariable('w', ['a'], ['initw','a']),
    MVariable('z', [], ['h'])
]
complicatedmodel = Model(v={x.label : x for x in modelVariables},
              o={x.label : x for x in modelOperations})

print('\n\n'+str(complicatedmodel)+'\n\n')
print('\n\n'+str(planCalc(complicatedmodel,VariablesSet(['n']),VariablesSet([('z',(0,10))])))+'\n\n')



{'n': n(entry:['initxn'], comp:[]), 'xn': xn(entry:['initw'], comp:['initxn']), 'x': x(entry:['f', 'h', 'initxn'], comp:['initx', 'f']), 'y': y(entry:['g', 'h', 'a'], comp:['f', 'g']), 'w': w(entry:['a'], comp:['initw', 'a']), 'z': z(entry:[], comp:['h'])}
{'initx': initx(ins:[], outs:[('x', (0, 0))]), 'f': f[{f[i+0]}](ins:[('x', (1, 0))], outs:[('x', (1, 1)), ('y', (2, 0))]), 'g': g[{g[i+0]}](ins:[('y', (2, 0))], outs:[('y', (2, 1))]), 'h': h[{h[i+0]}](ins:[('x', (1, 0)), ('y', (1, 0))], outs:[('z', (1, 0))]), 'initxn': initxn(ins:['n', ('x', (1, 0))], outs:['xn']), 'initw': initw(ins:['xn'], outs:[('w', (0, 10))]), 'a': a[{a[i+0]}](ins:[('y', (3, 0)), ('w', (1, 0))], outs:[('w', (1, 1))])}


initx can be applied, ('x', (0, 0)) was added to V_0
V_0 is {n, x[0]}
for {n, x[0]} ops are {'f', 'initxn', 'h'}
	{f[0]} added to F'_0

imitation start
	F'_0 is {f[0]}
		x[(0, 1)] added to V'_1 because ('f', (0, 0)) is in F'_0
		y[(0, 0)] added to V'_1 because ('f', (0, 0)) is in F'_0
	L_0 is {

In [33]:
modelOperations=[
    SOperation('initf1', [('f',(0,10))] , [('f',(0,1))]),
    SOperation('initf2', [('f',(0,11))] , [('f',(0,2))]),
    SOperation('initf10', [] , [('f',(0,10))]),
    SOperation('initf11', [] , [('f',(0,11))]),
    SOperation('inits', ['n'] , [('s',(0,0))]),
    MOperation('next', [('f',(1,1)),('f',(1,2))], [('f',(1,3))], VariablesSet([('next',(1,0))])),
    MOperation('sum', [('f',(1,1)),('s',(1,0))], [('s',(1,1))], VariablesSet([('sum',(1,0))])),
    SOperation('initfn', ['n',('f',(1,1))], ['fn']),
    SOperation('initsn', ['n',('s',(1,0))], ['sn'])
    ]
modelVariables=[
    SVariable('n', ['inits','initfn','initsn'], []),
    SVariable('fn', [], ['initfn']),
    SVariable('sn', [], ['initsn']),
    MVariable('f', ['initf1','initf2','next', 'sum','initfn'], ['initf1','initf2','initf10','initf11','next']),
    MVariable('s', ['sum','initsn'],['inits','sum'])
]
psiignored = Model(v={x.label : x for x in modelVariables},
              o={x.label : x for x in modelOperations})
print('\n\n'+str(psiignored)+'\n\n')
print('\n\n'+str(planCalc(psiignored,VariablesSet(['n']),VariablesSet(['fn',('s',(0,10))])))+'\n\n')



{'n': n(entry:['inits', 'initfn', 'initsn'], comp:[]), 'fn': fn(entry:[], comp:['initfn']), 'sn': sn(entry:[], comp:['initsn']), 'f': f(entry:['initf1', 'initf2', 'next', 'sum', 'initfn'], comp:['initf1', 'initf2', 'initf10', 'initf11', 'next']), 's': s(entry:['sum', 'initsn'], comp:['inits', 'sum'])}
{'initf1': initf1(ins:[('f', (0, 10))], outs:[('f', (0, 1))]), 'initf2': initf2(ins:[('f', (0, 11))], outs:[('f', (0, 2))]), 'initf10': initf10(ins:[], outs:[('f', (0, 10))]), 'initf11': initf11(ins:[], outs:[('f', (0, 11))]), 'inits': inits(ins:['n'], outs:[('s', (0, 0))]), 'next': next[{next[i+0]}](ins:[('f', (1, 1)), ('f', (1, 2))], outs:[('f', (1, 3))]), 'sum': sum[{sum[i+0]}](ins:[('f', (1, 1)), ('s', (1, 0))], outs:[('s', (1, 1))]), 'initfn': initfn(ins:['n', ('f', (1, 1))], outs:['fn']), 'initsn': initsn(ins:['n', ('s', (1, 0))], outs:['sn'])}


initf10 can be applied, ('f', (0, 10)) was added to V_0
initf11 can be applied, ('f', (0, 11)) was added to V_0
V_0 is {n, f[10,11]}
for

In [34]:
modelOperations=[
    SOperation('e', ['n'] , [('x',(0,1))]),
    MOperation('a', [('x',(1,0))] , [('y',(1,0))], VariablesSet([('a',(1,0))])),
    MOperation('b', [('y',(1,0))] , [('x',(1,2))], VariablesSet([('b',(1,0))])),
    MOperation('c', [('y',(1,0))] , [('x',(1,3))], VariablesSet([('c',(1,0))]))
    ]
modelVariables=[
    SVariable('n', ['e'], []),
    MVariable('x', ['a'], ['e','b','c']),
    MVariable('y', ['b','c'], ['a'])
]
testmodel = Model(v={x.label : x for x in modelVariables},
              o={x.label : x for x in modelOperations})

print('\n\n'+str(testmodel)+'\n\n')
testmodelready4lunatranslation = planCalc(testmodel,VariablesSet(['n']),VariablesSet([('x',(0,20))]))
print('\n\n'+str(testmodelready4lunatranslation)+'\n\n')



{'n': n(entry:['e'], comp:[]), 'x': x(entry:['a'], comp:['e', 'b', 'c']), 'y': y(entry:['b', 'c'], comp:['a'])}
{'e': e(ins:['n'], outs:[('x', (0, 1))]), 'a': a[{a[i+0]}](ins:[('x', (1, 0))], outs:[('y', (1, 0))]), 'b': b[{b[i+0]}](ins:[('y', (1, 0))], outs:[('x', (1, 2))]), 'c': c[{c[i+0]}](ins:[('y', (1, 0))], outs:[('x', (1, 3))])}


V_0 is {n}
for {n} ops are {'e'}
	e added to F_0
F_0 is {e}
	x[(0, 1)] added to V_1 because e is in F_0
V_1 is {n, x[1]}
for {x[1]} ops are {'a'}
	{a[1]} added to F'_0

imitation start
	F'_0 is {a[1]}
		y[(0, 1)] added to V'_1 because ('a', (0, 1)) is in F'_0
	L_0 is {'a': [(0, 1)]}
	V'_1 is {n, x[1], y[1]}
	F'sum_1 is {a[1]}
new primeops are {'c', 'b'}
		{c[1]} added to F'_1
		{b[1]} added to F'_1
		x[(0, 4)] added to V'_2 because ('c', (0, 1)) is in F'_1
		x[(0, 3)] added to V'_2 because ('b', (0, 1)) is in F'_1
	L_1 is {'a': [(0, 1)], 'c': [(1, 1)], 'b': [(1, 1)]}
	V'_2 is {n, x[1,3,4], y[1]}
	F'sum_2 is {a[1], c[1], b[1]}
new primeops are {'a'}
		

In [35]:
modelOperations=[
    SOperation('e', ['n'] , [('x',(0,1))]),
    MOperation('a', [('x',(1,0))] , [('y',(2,0))], VariablesSet([('a',(1,0))])),
    MOperation('b', [('y',(1,0))] , [('x',(1,0))], VariablesSet([('b',(1,0))])),
    ]
modelVariables=[
    SVariable('n', ['e'], []),
    MVariable('x', ['a'], ['e','b',]),
    MVariable('y', ['b'], ['a'])
]
wrongmodel = Model(v={x.label : x for x in modelVariables},
              o={x.label : x for x in modelOperations})

print('\n\n'+str(wrongmodel)+'\n\n')
wrongmodelready4lunatranslation = planCalc(wrongmodel,VariablesSet(['n']),VariablesSet([('x',(0,20))]))
print('\n\n'+str(wrongmodelready4lunatranslation)+'\n\n')



{'n': n(entry:['e'], comp:[]), 'x': x(entry:['a'], comp:['e', 'b']), 'y': y(entry:['b'], comp:['a'])}
{'e': e(ins:['n'], outs:[('x', (0, 1))]), 'a': a[{a[i+0]}](ins:[('x', (1, 0))], outs:[('y', (2, 0))]), 'b': b[{b[i+0]}](ins:[('y', (1, 0))], outs:[('x', (1, 0))])}


V_0 is {n}
for {n} ops are {'e'}
	e added to F_0
F_0 is {e}
	x[(0, 1)] added to V_1 because e is in F_0
V_1 is {n, x[1]}
for {x[1]} ops are {'a'}
	{a[1]} added to F'_0

imitation start
	F'_0 is {a[1]}
		y[(0, 2)] added to V'_1 because ('a', (0, 1)) is in F'_0
	L_0 is {'a': [(0, 1)]}
	V'_1 is {n, x[1], y[2]}
	F'sum_1 is {a[1]}
new primeops are {'b'}
		{b[2]} added to F'_1
		x[(0, 2)] added to V'_2 because ('b', (0, 2)) is in F'_1
	L_1 is {'a': [(0, 1)], 'b': [(1, 2)]}
	V'_2 is {n, x[1,2], y[2]}
	F'sum_2 is {a[1], b[2]}
new primeops are {'a'}
		{a[2]} added to F'_2
		trying to build terms for a: (0, 1) and (2, 2)
building terms for a: 0,1 and 2,2
a[(1, 1)](x[(1, 1)]) and a[(1, 2)](x[(1, 2)]) are identical with step = 1
new

In [1]:
GAUSS_N = 17

In [36]:
modelOperations=[
    MOperation('init', [('init',(1,0))] , [('x',(1,0))], VariablesSet([('init',(1,0))]).difference(VariablesSet([('init',(1,GAUSS_N))]))),
    MOperation('blurleft', [('x',(1,-GAUSS_N)),('x',(1,1-GAUSS_N))] , [('x',(1,0))], VariablesSet([('blurleft',(GAUSS_N,GAUSS_N))])),
    MOperation('blurright', [('x',(1,-1)),('x',(1,0))] , [('x',(1,GAUSS_N))], VariablesSet([('blurright',(GAUSS_N,GAUSS_N-1))])),
    MOperation('blurmiddle', [('x',(1,-1)),('x',(1,0)),('x',(1,1))] , [('x',(1,GAUSS_N))],
               VariablesSet([('blurmiddle',(1,0))]).difference(VariablesSet([('blurmiddle',(GAUSS_N,0))]).union(VariablesSet([('blurmiddle',(GAUSS_N,GAUSS_N-1))])))),
    MOperation('sum0', [('x',(1,0))] , [('s',(1,0))], VariablesSet([('sum0',(GAUSS_N,0))])),
    MOperation('sum', [('x',(1,0)),('s',(1,-1))] , [('s',(1,0))],
               VariablesSet([('sum',(1,0))]).difference(VariablesSet([('sum',(GAUSS_N,0))]))),
    MOperation('diff', [('s',(GAUSS_N,GAUSS_N-1)),('s',(GAUSS_N,-1))], [('d',(1,1))], VariablesSet([('diff',(1,1))])),
    SOperation('getsn', ['n', ('s',(GAUSS_N,GAUSS_N-1))] , ['sn']),
    SOperation('getdn', ['n', ('d',(1,2))] , ['dn'])
    ]
modelVariables=[
    MVariable('init', ['init'], []),
    SVariable('n', ['getsn'], []),
    SVariable('sn', [], ['getsn']),
    SVariable('dn', [], ['getdn']),
    MVariable('x', ['blurleft','blurright','blurmiddle','sum0','sum'], ['blurleft','blurright','blurmiddle','init']),
    MVariable('s', ['sum','sum0','getsn','diff'], ['sum','sum0']),
    MVariable('d', ['getdn'], ['diff'])
]
gaussmodel = Model(v={x.label : x for x in modelVariables},
              o={x.label : x for x in modelOperations})

print('\n\n'+str(gaussmodel)+'\n\n')
gaussmodelready4lunatranslation = planCalc(gaussmodel,
                                           VariablesSet(['n',('init',(1,0))]).difference(VariablesSet([('init',(1,GAUSS_N))])),
                                           VariablesSet(['dn']), maxprimelevel=30)
print('\n\n'+str(gaussmodelready4lunatranslation)+'\n\n')



{'init': init(entry:['init'], comp:[]), 'n': n(entry:['getsn'], comp:[]), 'sn': sn(entry:[], comp:['getsn']), 'dn': dn(entry:[], comp:['getdn']), 'x': x(entry:['blurleft', 'blurright', 'blurmiddle', 'sum0', 'sum'], comp:['blurleft', 'blurright', 'blurmiddle', 'init']), 's': s(entry:['sum', 'sum0', 'getsn', 'diff'], comp:['sum', 'sum0']), 'd': d(entry:['getdn'], comp:['diff'])}
{'init': init[{init[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]}](ins:[('init', (1, 0))], outs:[('x', (1, 0))]), 'blurleft': blurleft[{blurleft[17i+17]}](ins:[('x', (1, -17)), ('x', (1, -16))], outs:[('x', (1, 0))]), 'blurright': blurright[{blurright[17i+16]}](ins:[('x', (1, -1)), ('x', (1, 0))], outs:[('x', (1, 17))]), 'blurmiddle': blurmiddle[{blurmiddle[17i+1], blurmiddle[17i+2], blurmiddle[17i+3], blurmiddle[17i+4], blurmiddle[17i+5], blurmiddle[17i+6], blurmiddle[17i+7], blurmiddle[17i+8], blurmiddle[17i+9], blurmiddle[17i+10], blurmiddle[17i+11], blurmiddle[17i+12], blurmiddle[17i+13], blurmiddle[17i+14], b

KeyboardInterrupt: 

In [37]:
modelOperations=[
    SOperation('e', ['n'] , [('x',(0,1)),('y',(0,1))]),
    MOperation('a', [('x',(1,0)),('y',(1,0))] , [('x',(1,1))], VariablesSet([('a',(1,0))])),
    MOperation('b', [('x',(1,0)),('y',(1,0))] , [('y',(1,1))], VariablesSet([('b',(1,0))]))
    ]
modelVariables=[
    SVariable('n', ['e'], []),
    MVariable('x', ['a','b'], ['e','a',]),
    MVariable('y', ['a','b'], ['e','b'])
]
newalgnmodel = Model(v={x.label : x for x in modelVariables},
              o={x.label : x for x in modelOperations})

print('\n\n'+str(newalgnmodel)+'\n\n')
newalgnmodelready4lunatranslation = planCalc(newalgnmodel,VariablesSet(['n']),VariablesSet([('x',(0,20))]))
print('\n\n'+str(newalgnmodelready4lunatranslation)+'\n\n')



{'n': n(entry:['e'], comp:[]), 'x': x(entry:['a', 'b'], comp:['e', 'a']), 'y': y(entry:['a', 'b'], comp:['e', 'b'])}
{'e': e(ins:['n'], outs:[('x', (0, 1)), ('y', (0, 1))]), 'a': a[{a[i+0]}](ins:[('x', (1, 0)), ('y', (1, 0))], outs:[('x', (1, 1))]), 'b': b[{b[i+0]}](ins:[('x', (1, 0)), ('y', (1, 0))], outs:[('y', (1, 1))])}


V_0 is {n}
for {n} ops are {'e'}
	e added to F_0
F_0 is {e}
	x[(0, 1)] added to V_1 because e is in F_0
	y[(0, 1)] added to V_1 because e is in F_0
V_1 is {n, x[1], y[1]}
for {x[1], y[1]} ops are {'b', 'a'}
	{b[1]} added to F'_0
	{a[1]} added to F'_0

imitation start
	F'_0 is {b[1], a[1]}
		y[(0, 2)] added to V'_1 because ('b', (0, 1)) is in F'_0
		x[(0, 2)] added to V'_1 because ('a', (0, 1)) is in F'_0
	L_0 is {'b': [(0, 1)], 'a': [(0, 1)]}
	V'_1 is {n, x[1,2], y[1,2]}
	F'sum_1 is {b[1], a[1]}
new primeops are {'b', 'a'}
		{b[2]} added to F'_1
		{a[2]} added to F'_1
		trying to build terms for b: (0, 1) and (1, 2)
building terms for b: 0,1 and 1,2
b[(1, 1)](x[

Генерация LuNA (в процессе)

In [38]:
def var2luna(srcModel:Model, model:Model, v, ready:VariablesSet, V:VariablesSet, border = 0):

    print(f'var2luna {v}')
    if v in ready or v in V:
        return ''
    lunaVar = ''
    if isinstance(v, tuple):
            for opl in model.v[v[0]].comp:
                op = model.o[opl]
                toComp = VariablesSet([v]).difference(ready)
                if toComp.empty():
                     break
                res = canApplyBackward(op, toComp)
                for r in res:
                    lunaVar += op2luna(srcModel, model, r, ready, V, v)
    else:
            for opl in model.v[v].comp:
                op = model.o[opl]
                if isinstance(op, MOperation):
                    pass
                else:
                    lunaVar += op2luna(srcModel, model, opl, ready, V, v)
    return lunaVar

In [39]:
def findPsiUsageLevel(usage, model, var):
    vl = var[0]
    for u in usage:
        operation = model.o[u[0]]
        for ov in operation.outs:
            if not isinstance(ov,tuple):
                continue
            if vl == ov[0]:
                descVar = (vl,(u[1][0]*ov[1][0], u[1][1]*ov[1][0]+ov[1][1]))
                if var[1][1]>=descVar[1][1] and (var[1][1]-descVar[1][1]) % descVar[1][0] == 0:
                    t = (var[1][1]-descVar[1][1]) // descVar[1][0]
                    return t
    return None

In [40]:
def generatePsi(usage, model:Model, shift, delim=''):
    result = ''
    for u in usage:
        term = (u[0],(u[1][0],u[1][0]*shift+u[1][1]))
        operation = model.o[u[0]]
        result += delim + operation.label + '(' +','.join([ (x[0]+ '[' + (('' if t == 1 else str(t)+'*') + 'i+' if (t:=x[1][0]*term[1][0]) >= 1 else ''  )+ str(x[1][0]*term[1][1]+x[1][1])+']') if isinstance(x,tuple) else x for x in operation.ins] +
                                                          [ (x[0]+ '[' + (('' if t == 1 else str(t)+'*') + 'i+' if (t:=x[1][0]*term[1][0]) >= 1 else ''  )+ str(x[1][0]*term[1][1]+x[1][1])+']') if isinstance(x,tuple) else x for x in operation.outs]
                                                          ) + ');\n'
    return result

def generatePsiSimple(usage, model:Model, shift, delim=''):
    result = ''
    for u in usage:
        term = (u[0],(u[1][0],u[1][0]*shift+u[1][1]))
        operation = model.o[u[0]]
        result += delim + operation.label + '(' +','.join([ (x[0]+ '[' + str(x[1][0]*term[1][1]+x[1][1])+']') if isinstance(x,tuple) else x for x in operation.ins] +
                                                          [ (x[0]+ '[' + str(x[1][0]*term[1][1]+x[1][1])+']') if isinstance(x,tuple) else x for x in operation.outs]
                                                          ) + ');\n'
    return result

In [41]:
def usageOutput(usage, model:Model, shift):
    result = VariablesSet()
    for u in usage:
        term = (u[0],(u[1][0],u[1][0]*shift+u[1][1]))
        operation = model.o[u[0]]
        for x in operation.outs:
            result.addItem( (x[0],(0,x[1][0]*term[1][1]+x[1][1])) if isinstance(x,tuple) else x)
    return result

In [42]:
def psiDependencies(srcModel:Model, model:Model, ready:VariablesSet, V:VariablesSet, infvars:VariablesSet, usage, i):
    res = ''
    for u in usage:
        for iv in srcModel.o[u[0]].ins:
            border = (iv[0],(0,iv[1][0]*i+iv[1][1]))
            #for infv in VariablesSet([iv]).intersection(infvars):
            res += var2luna(srcModel, model, border, ready, V)
    return res
        

In [43]:
def op2luna(srcModel:Model, model:Model, op, ready:VariablesSet, V:VariablesSet, var):

    print(f'op2luna {op}, (var = {var})')
    if isinstance(op, tuple):
        if op[1][0] != 0:
            print('plan is invalid')
        opl = op[0]
        opi = op[1][1]
        operation = model.o[opl]
        lunaOp = ''
        for iv in operation.ins:
            v = (iv[0],(0,iv[1][0]*opi+iv[1][1])) if isinstance(iv, tuple) else iv
            lunaOp += var2luna(srcModel, model, v, ready, V)
        opargs = ', '.join([(str(x[1][0]*opi+x[1][1]) if x[0]==opl else (x[0]+'['+str(x[1][0]*opi+x[1][1])+']')) if isinstance(x,tuple) else x for x in operation.ins]
                        +  [ ( '_' if ready.isinValue(t:= (x[0],(0,x[1][0]*opi+x[1][1]))) else t[0]+'['+str(t[1][1])+']') if isinstance(x,tuple) else x for x in operation.outs])
        for ov in operation.outs:
            v = (ov[0],(0,ov[1][0]*opi+ov[1][1])) if isinstance(ov, tuple) else ov
            ready.addItem(v)
        return lunaOp + operation.label + '('+ opargs +');\n'
    else:
        lunaOp = ''
        operation = model.o[op]
        if operation.psi == None:
            for iv in operation.ins:
                lunaOp += var2luna(srcModel, model, iv, ready, V)
            opargs = ', '.join([x[0]+'['+str(x[1][1])+']' if isinstance(x,tuple) else x for x in operation.ins]
                        +  [ ( '_' if ready.isinValue(x) else x[0]+'['+str(x[1][1])+']') if isinstance(x,tuple) else x for x in operation.outs])
            for ov in operation.outs:
                ready.addItem(ov)
            return lunaOp + operation.label + '('+ opargs +');\n'
        else:
            if isinstance(var,tuple) and var[1][0]==0:
                mini = findPsiUsageLevel(operation.usage, srcModel, var)
                print(f'mini is {mini}')
                if mini == None:
                    return ''
                infvars = VariablesSet()
                if operation.auto:
                    for iv in termIns(operation.psi, mini):
                        lunaOp += var2luna(srcModel, model, iv, ready, V)
                    psibody = generatePsiSimple(operation.usage, srcModel, mini)
                    ready.update(usageOutput(operation.usage, srcModel, mini))
                    return lunaOp + psibody
                else:
                    for iv in operation.ins:
                        if isinstance(iv,tuple) and iv[1][0] == 0:
                            lunaOp += var2luna(srcModel, model, iv, ready, V)
                        else:
                            infvars.addItem(iv)
                    lunaOp += psiDependencies(srcModel, model, ready, V, infvars, operation.usage, mini)

                    operation.scope = mini if operation.scope is None else max(operation.scope, mini)

                    ready.update(termOutput(operation.psi, srcModel, mini))
                    return lunaOp
            else:
                return ''

In [44]:
def model2luna(srcModel:Model, model:Model, V:VariablesSet, W:VariablesSet):
    ready = copy.deepcopy(V)
    lunaModel = ''
    for v in W:
        lunaModel += var2luna(srcModel, model, v, ready, V)
    
    for opl, op in model.o.items():
        if isinstance(op, SOperation) and op.psi is not None and op.auto == False:
            lunaModel += f'for i = {0}..{op.scope}\n' + '{\n' + generatePsi(op.usage, srcModel, 0, delim='\t') + '}\n'


    external = V.union(W)
    declarations = [x for x in ready.s if x not in external.s]+[x+'['+']' for x in ready.m if x not in external.m]
    if len(declarations) == 0:
        return '{\n' + lunaModel + '}'
    else:
        return '{\nvar '+', '.join(declarations) + ';\n' + lunaModel + '}'
            

In [99]:
modelOperations=[
    SOperation('initx', ['n'] , [('x',(0,0)),('y',(0,1))]),
    MOperation('f', [('x',(1,0))] , [('x',(1,1)),('y',(1,0))], VariablesSet([('f',(1,0))]).difference(VariablesSet([('f',(1,11))])))
    ]
modelVariables=[
    SVariable('n', ['initx'], []),
    MVariable('x', ['f'], ['initx','f']),
    MVariable('y', [], ['f'])
]
plannedmodel = Model(v={x.label : x for x in modelVariables},
              o={x.label : x for x in modelOperations})

print('\n'+model2luna(plannedmodel, plannedmodel, VariablesSet(), VariablesSet([('x',(0,9))])))

var2luna ('x', (0, 9))
op2luna ('f', (0, 8)), (var = ('x', (0, 9)))
var2luna ('x', (0, 8))
op2luna ('f', (0, 7)), (var = ('x', (0, 8)))
var2luna ('x', (0, 7))
op2luna ('f', (0, 6)), (var = ('x', (0, 7)))
var2luna ('x', (0, 6))
op2luna ('f', (0, 5)), (var = ('x', (0, 6)))
var2luna ('x', (0, 5))
op2luna ('f', (0, 4)), (var = ('x', (0, 5)))
var2luna ('x', (0, 4))
op2luna ('f', (0, 3)), (var = ('x', (0, 4)))
var2luna ('x', (0, 3))
op2luna ('f', (0, 2)), (var = ('x', (0, 3)))
var2luna ('x', (0, 2))
op2luna ('f', (0, 1)), (var = ('x', (0, 2)))
var2luna ('x', (0, 1))
op2luna ('f', (0, 0)), (var = ('x', (0, 1)))
var2luna ('x', (0, 0))
op2luna initx, (var = ('x', (0, 0)))
var2luna n

{
var y[];
initx(n, x[0], y[1]);
f(x[0], x[1], y[0]);
f(x[1], x[2], _);
f(x[2], x[3], y[2]);
f(x[3], x[4], y[3]);
f(x[4], x[5], y[4]);
f(x[5], x[6], y[5]);
f(x[6], x[7], y[6]);
f(x[7], x[8], y[7]);
f(x[8], x[9], y[8]);
}


In [100]:
print('\n'+model2luna(simplemodel, simplemodelready4lunatranslation, VariablesSet(['n']), VariablesSet([('z',(0,20))])))

var2luna ('z', (0, 20))
op2luna psi1, (var = ('z', (0, 20)))
mini is 17
var2luna ('y', (0, 20))
op2luna psi0, (var = ('y', (0, 20)))
mini is 19
var2luna ('x', (0, 1))
op2luna e, (var = ('x', (0, 1)))
var2luna n
var2luna ('x', (0, 19))
op2luna psi0, (var = ('x', (0, 19)))
mini is 17
var2luna ('x', (0, 1))
var2luna ('x', (0, 17))
op2luna psi0, (var = ('x', (0, 17)))
mini is 15
var2luna ('x', (0, 1))
var2luna ('x', (0, 15))
op2luna psi0, (var = ('x', (0, 15)))
mini is 13
var2luna ('x', (0, 1))
var2luna ('x', (0, 13))
op2luna psi0, (var = ('x', (0, 13)))
mini is 11
var2luna ('x', (0, 1))
var2luna ('x', (0, 11))
op2luna psi0, (var = ('x', (0, 11)))
mini is 9
var2luna ('x', (0, 1))
var2luna ('x', (0, 9))
op2luna psi0, (var = ('x', (0, 9)))
mini is 7
var2luna ('x', (0, 1))
var2luna ('x', (0, 7))
op2luna psi0, (var = ('x', (0, 7)))
mini is 5
var2luna ('x', (0, 1))
var2luna ('x', (0, 5))
op2luna psi0, (var = ('x', (0, 5)))
mini is 3
var2luna ('x', (0, 1))
var2luna ('x', (0, 3))
op2luna psi0, (v

In [101]:
print('\n'+model2luna(fibmodel, fibmodelready4lunatranslation, VariablesSet(['n']), VariablesSet([('s',(0,9))])))

var2luna ('s', (0, 9))
op2luna psi1, (var = ('s', (0, 9)))
mini is 8
var2luna ('s', (0, 0))
op2luna inits, (var = ('s', (0, 0)))
var2luna ('f', (0, 9))
op2luna psi0, (var = ('f', (0, 9)))
mini is 6
var2luna ('f', (0, 1))
op2luna initf1, (var = ('f', (0, 1)))
var2luna ('f', (0, 2))
op2luna initf2, (var = ('f', (0, 2)))
var2luna ('f', (0, 7))
op2luna psi0, (var = ('f', (0, 7)))
mini is 4
var2luna ('f', (0, 1))
var2luna ('f', (0, 2))
var2luna ('f', (0, 5))
op2luna psi0, (var = ('f', (0, 5)))
mini is 2
var2luna ('f', (0, 1))
var2luna ('f', (0, 2))
var2luna ('f', (0, 3))
op2luna psi0, (var = ('f', (0, 3)))
mini is 0
var2luna ('f', (0, 1))
var2luna ('f', (0, 2))
var2luna ('f', (0, 1))
var2luna ('f', (0, 2))
var2luna ('f', (0, 4))
op2luna psi0, (var = ('f', (0, 4)))
mini is 1
var2luna ('f', (0, 1))
var2luna ('f', (0, 2))
var2luna ('f', (0, 2))
var2luna ('f', (0, 3))
var2luna ('f', (0, 6))
op2luna psi0, (var = ('f', (0, 6)))
mini is 3
var2luna ('f', (0, 1))
var2luna ('f', (0, 2))
var2luna ('f'

In [45]:
print('\n'+model2luna(gaussmodel, gaussmodelready4lunatranslation,
                      VariablesSet(['n',('init',(1,0))]).difference(VariablesSet([('init',(1,GAUSS_N))])),
                      VariablesSet([('d',(0,15))])))

var2luna ('d', (0, 15))
op2luna psi6, (var = ('d', (0, 15)))
mini is 10
var2luna ('s', (0, 74))
op2luna psi5, (var = ('s', (0, 74)))
mini is 10
var2luna ('x', (0, 74))
op2luna psi0, (var = ('x', (0, 74)))
mini is 13
var2luna ('x', (0, 0))
op2luna ('init', (0, 0)), (var = ('x', (0, 0)))
var2luna ('init', (0, 0))
var2luna ('x', (0, 1))
op2luna ('init', (0, 1)), (var = ('x', (0, 1)))
var2luna ('init', (0, 1))
var2luna ('x', (0, 2))
op2luna ('init', (0, 2)), (var = ('x', (0, 2)))
var2luna ('init', (0, 2))
var2luna ('x', (0, 3))
op2luna ('init', (0, 3)), (var = ('x', (0, 3)))
var2luna ('init', (0, 3))
var2luna ('x', (0, 4))
op2luna ('init', (0, 4)), (var = ('x', (0, 4)))
var2luna ('init', (0, 4))
var2luna ('x', (0, 8))
op2luna psi0, (var = ('x', (0, 8)))
mini is 0
var2luna ('x', (0, 0))
var2luna ('x', (0, 1))
var2luna ('x', (0, 2))
var2luna ('x', (0, 3))
var2luna ('x', (0, 4))
var2luna ('x', (0, -5))
var2luna ('x', (0, -4))
var2luna ('x', (0, -1))
var2luna ('x', (0, 0))
var2luna ('x', (0, 1

In [50]:
gaussmodelready4lunatranslation.o['psi0'].usage

[('blurleft', (5, 5)),
 ('blurmiddle', (5, 1)),
 ('blurmiddle', (5, 2)),
 ('blurmiddle', (5, 3)),
 ('blurright', (5, 4))]